In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import numpy as np
import matplotlib.pyplot as plt

# Usamos el estilo "ggplot", que suele estar disponible.
plt.style.use('ggplot')
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'legend.fontsize': 12,
    'figure.figsize': (8, 6),
    'lines.linewidth': 2,
    'grid.linestyle': '--',
    'grid.alpha': 0.6
})

# Carpeta para guardar las figuras.
FIGURES_DIR = "figuras"
os.makedirs(FIGURES_DIR, exist_ok=True)

def read_xvg(filename):
    """
    Lee un archivo .xvg de Gromacs, ignorando líneas que empiecen por '#' o '@'.
    Devuelve un array de shape (N, M) con los datos numéricos.
    """
    data = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#') or line.startswith('@'):
                continue
            parts = line.split()
            data.append([float(x) for x in parts])
    return np.array(data)

def read_dat(filename):
    """
    Lee un archivo .dat genérico con columnas numéricas.
    Devuelve un array de shape (N, M).
    """
    data = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            data.append([float(x) for x in parts])
    return np.array(data)

def plot_xvg_2col(data, x_label, y_label, title, outname):
    """
    Grafica un archivo .xvg (2 columnas: x, y) en forma de línea.
    """
    x = data[:, 0]
    y = data[:, 1]
    plt.figure()
    plt.plot(x, y, color='teal')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title + " (400K)")
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def plot_histogram(data, col=1, bins=50, x_label='', y_label='Frecuencia', title='', outname='hist.png'):
    """
    Grafica un histograma de la columna 'col' del array 'data'.
    """
    values = data[:, col]
    plt.figure()
    plt.hist(values, bins=bins, color='skyblue', edgecolor='black')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title + " (400K)")
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def plot_single_angle(data, angle_name='phi', outname='angle.png'):
    """
    Grafica un ángulo (archivo .dat con 2 columnas: frame y valor) en función del frame.
    """
    frames = data[:, 0]
    angle = data[:, 1]
    plt.figure()
    plt.plot(frames, angle, marker='o', markersize=4, linestyle='-', color='darkorange')
    plt.xlabel('Frame')
    plt.ylabel(f'{angle_name} (grados)')
    plt.title(f'Evolución de {angle_name} (400K)')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def plot_ramachandran_2d_dual(data1, label1, data2, label2, outname):
    """
    Genera un diagrama de Ramachandran combinando dos conjuntos de datos.
    Cada conjunto (data1 y data2) debe ser un array con 3 columnas: (índice, phi, psi).
    Se utilizan colores distintos y se añade una leyenda indicando qué color corresponde a cada aminoácido.
    """
    plt.figure()
    # Primer conjunto en azul.
    plt.scatter(data1[:, 1], data1[:, 2], s=20, c='blue', alpha=0.7, edgecolors='w', label=label1)
    # Segundo conjunto en rojo.
    plt.scatter(data2[:, 1], data2[:, 2], s=20, c='red', alpha=0.7, edgecolors='w', label=label2)
    plt.xlabel(r'$\phi$ (grados)')
    plt.ylabel(r'$\psi$ (grados)')
    plt.title('Diagrama de Ramachandran (400K)')
    plt.xlim([-180, 180])
    plt.ylim([-180, 180])
    plt.xticks(np.arange(-180, 181, 60))
    plt.yticks(np.arange(-180, 181, 60))
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def plot_velocity_histograms(data, outname='veloc_hist.png'):
    """
    Asume que 'data' es un array con al menos 4 columnas:
    tiempo, vx, vy, vz.
    Crea histogramas separados para cada componente de la velocidad.
    """
    vx = data[:, 1]
    vy = data[:, 2]
    vz = data[:, 3] if data.shape[1] > 3 else None

    ncols = 3 if vz is not None else 2
    fig, axs = plt.subplots(1, ncols, figsize=(5*ncols, 5))
    axs[0].hist(vx, bins=50, color='salmon', edgecolor='black')
    axs[0].set_xlabel('vx (nm/ps)')
    axs[0].set_ylabel('Frecuencia')
    axs[0].set_title('Evolución de vx (400K)')
    
    axs[1].hist(vy, bins=50, color='lightgreen', edgecolor='black')
    axs[1].set_xlabel('vy (nm/ps)')
    axs[1].set_ylabel('Frecuencia')
    axs[1].set_title('Evolución de vy (400K)')
    
    if vz is not None:
        axs[2].hist(vz, bins=50, color='lightblue', edgecolor='black')
        axs[2].set_xlabel('vz (nm/ps)')
        axs[2].set_ylabel('Frecuencia')
        axs[2].set_title('Evolución de vz (400K)')
    
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, outname), dpi=300)
    plt.close()

def main():
    """
    Función principal que llama a las funciones de graficado para cada archivo.
    Se añade "_400k" al final del nombre de archivo de salida y " (400K)" en el título.
    Ajusta los nombres de archivo según corresponda.
    """
    # GYRATE: Evolución del radio de giro (nm)
    gyrate_data = read_xvg('gyrate.xvg')
    plot_xvg_2col(gyrate_data,
                  x_label='Tiempo (ps)',
                  y_label='Radio de giro (nm)',
                  title='Evolución del radio de giro',
                  outname='gyrate_400k.png')
    
    # DISTANCIAS:
    # [CA-CB] (ala-2)
    dist1_data = read_xvg('dist1.xvg')
    plot_xvg_2col(dist1_data,
                  x_label='Tiempo (ps)',
                  y_label='Distancia (nm)',
                  title='Evolución de la distancia [CA-CB] (ala-2)',
                  outname='dist1_400k.png')
    
    # [N-CA] (gly-4)
    dist2_data = read_xvg('dist2.xvg')
    plot_xvg_2col(dist2_data,
                  x_label='Tiempo (ps)',
                  y_label='Distancia (nm)',
                  title='Evolución de la distancia [N-CA] (gly-4)',
                  outname='dist2_400k.png')
    
    # ÁNGULOS PROMEDIO:
    # angaver1.xvg corresponde a [CA-CB-CG] (arg-3)
    angaver1_data = read_xvg('angaver1.xvg')
    plot_xvg_2col(angaver1_data,
                  x_label='Tiempo (ps)',
                  y_label='Ángulo (grados)',
                  title='Evolución del ángulo [CA-CB-CG] (arg-3)',
                  outname='angaver1_400k.png')
    
    # angaver2.xvg corresponde a [CA-C-O] (gly-4)
    angaver2_data = read_xvg('angaver2.xvg')
    plot_xvg_2col(angaver2_data,
                  x_label='Tiempo (ps)',
                  y_label='Ángulo (grados)',
                  title='Evolución del ángulo [CA-C-O] (gly-4)',
                  outname='angaver2_400k.png')
    
    # ÁNGULOS: psi y phi de ALA-2 (archivos separados)
    phi_ala2 = read_dat('phi-ala-2.dat')
    plot_single_angle(phi_ala2, angle_name='phi (ALA-2)', outname='phi_ala2_400k.png')
    
    psi_ala2 = read_dat('psi-ala-2.dat')
    plot_single_angle(psi_ala2, angle_name='psi (ALA-2)', outname='psi_ala2_400k.png')
    
    # ÁNGULOS: psi y phi de ARG-3 (archivos separados)
    phi_arg3 = read_dat('phi-arg-3.dat')
    plot_single_angle(phi_arg3, angle_name='phi (ARG-3)', outname='phi_arg3_400k.png')
    
    psi_arg3 = read_dat('psi-arg-3.dat')
    plot_single_angle(psi_arg3, angle_name='psi (ARG-3)', outname='psi_arg3_400k.png')
    
    # DIAGRAMA DE RAMACHANDRAN COMBINADO:
    # Para ALA-2 y ARG-3 (archivos con 3 columnas: índice, phi, psi)
    ala2_dat = read_dat('ala-2.dat')
    arg3_dat = read_dat('arg-3.dat')
    plot_ramachandran_2d_dual(ala2_dat, 'ALA-2', arg3_dat, 'ARG-3', outname='rama_dual_400k.png')
    
    # TEMPERATURA: Histograma de temperatura (K)
    temp_data = read_xvg('temp.xvg')
    plot_histogram(temp_data,
                   col=1,
                   bins=50,
                   x_label='Temperatura (K)',
                   y_label='Frecuencia',
                   title='Histograma de temperatura',
                   outname='temp_hist_400k.png')
    
    # VELOCIDADES: Histograma para cada componente (vx, vy, vz)
    veloc_data = read_xvg('veloc.xvg')
    plot_velocity_histograms(veloc_data, outname='veloc_hist_400k.png')
    
    # ENERGÍA: Evolución de la energía (kJ/mol)
    energy_data = read_xvg('energy.xvg')
    plot_xvg_2col(energy_data,
                  x_label='Tiempo (ps)',
                  y_label='Energía (kJ/mol)',
                  title='Evolución de la energía',
                  outname='energy_400k.png')
    
    print("¡Gráficas generadas y guardadas en la carpeta 'figuras' con éxito!")

if __name__ == '__main__':
    main()


¡Gráficas generadas y guardadas en la carpeta 'figuras' con éxito!
